In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

# 1. Load the FashionMNIST data

In [2]:
transform= transforms.Compose(
    [
        transforms.ToTensor()
    ]
)

In [3]:
batch_size = 64

In [4]:
trainset = torchvision.datasets.FashionMNIST(root='./data',train=True,
                                       download=True, transform=transform)

100%|██████████████████████████████████████████████████████████████████| 26421880/26421880 [02:20<00:00, 188249.52it/s]


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 49024.76it/s]


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100%|████████████████████████████████████████████████████████████████████| 4422102/4422102 [00:16<00:00, 271964.89it/s]


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<?, ?it/s]

Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



In [7]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True)

In [8]:
len(trainset) # 60,000 images

60000

In [9]:
len(trainloader) # 60,000/64= 938 batches

938

In [10]:
testset = torchvision.datasets.FashionMNIST(root='./data',train=False,
                                       download=True, transform=transform)

In [11]:
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

In [12]:
len(testloader) #157 batches

157

In [13]:
len(testset) # 10,000 images

10000

In [14]:
for image,label in trainloader:
    print(image.shape)
    print(label.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [15]:
for image,label in testloader:
    print(image.shape)
    print(label.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


# 2. Define a Neural Network

In [16]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [18]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        self.fc1= nn.Linear(28*28, 512) # 28*28 from image dimension
        self.fc2= nn.Linear(512,128)
        self.fc3= nn.Linear(128,10)
        
    def forward(self, input):
        x= torch.flatten(input,1)
        x=self.fc1(x)
        x=F.relu(x)
        x=self.fc2(x)
        x=F.relu(x)
        x=self.fc3(x)
        return x
        
        

In [19]:
model= Net()
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
)


# 3. Define a Loss function and optimizer

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.001)

# 4. Training loop of the network

In [27]:
size = len(trainloader.dataset)
size

60000

In [42]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) #total number of image
    model.train()
    
    for batch, (image, label) in enumerate(dataloader):
        
        pred = model(image)
        loss = loss_fn(pred, label)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(image)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# 5. Testing loop of the network

In [43]:
size = len(testloader.dataset)
size

10000

In [44]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    total_loss, correct = 0, 0
    
    with torch.no_grad():
        for image, label in dataloader:
            pred = model(image)
            total_loss += loss_fn(pred, label).item()
            correct += (pred.argmax(1) == label).type(torch.float).sum().item()
    
    total_loss /= num_batches
    correct /= size
    print(f"Test result: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {total_loss:>8f} \n")

# 6. Model Evaluation

In [45]:
for epoch in range(1,6):
    print(f"Epoch {epoch}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 1.100297  [   64/60000]
loss: 0.971231  [ 6464/60000]
loss: 1.016848  [12864/60000]
loss: 0.991942  [19264/60000]
loss: 1.180007  [25664/60000]
loss: 1.121896  [32064/60000]
loss: 1.046325  [38464/60000]
loss: 0.943186  [44864/60000]
loss: 1.133814  [51264/60000]
loss: 0.961872  [57664/60000]
Test result: 
 Accuracy: 65.6%, Avg loss: 0.986796 

Epoch 2
-------------------------------
loss: 1.057878  [   64/60000]
loss: 1.017307  [ 6464/60000]
loss: 0.840443  [12864/60000]
loss: 0.900234  [19264/60000]
loss: 1.010291  [25664/60000]
loss: 0.911125  [32064/60000]
loss: 1.155897  [38464/60000]
loss: 0.939221  [44864/60000]
loss: 0.844992  [51264/60000]
loss: 0.924779  [57664/60000]
Test result: 
 Accuracy: 66.6%, Avg loss: 0.913504 

Epoch 3
-------------------------------
loss: 0.904804  [   64/60000]
loss: 0.809752  [ 6464/60000]
loss: 0.873153  [12864/60000]
loss: 0.951148  [19264/60000]
loss: 0.901934  [25664/60000]
loss: 1.076858  [32064/6

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")